In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt

In [ ]:
def load_waveform(fname):
    data = sio.loadmat(fname)
    k = list(data.keys())[-1]
    data = data[k]
    return pd.DataFrame(dict(
        fname=[x[0] for x in data['fname'][0]],
        marker=[x[0][0] for x in data['marker'][0]],
        sampling_rate=[x[0][0] for x in data['samprate'][0]],
        resistance=[x[0][0] for x in data['RO'][0]],
        capacitance=[x[0][0] for x in data['CO'][0]],
        amplitude_modulation=[x[0][0] for x in data['amp_mod'][0]],
        waveform_modulation=[x[0][0] for x in data['wav_mod'][0]],
        value_max=[x[0][0] for x in data['maxv'][0]],
        value_min=[x[0][0] for x in data['minv'][0]],
        waveform=[x[0] for x in data['waveform'][0]],
    ))
    

def load_lfp_data(fname):
    if fname[-4:] == ".mat":
        fname = "-".join(fname.split("-")[:-1])
    data_means = mat73.loadmat(fname + "-lfp_means.mat")["LfpMeans"]
    data_stats = mat73.loadmat(fname + "-lfp_stats.mat")["LfpStats"]

    lfp_means_time = data_means["lfptime"][-1]
    lfp_sampling_rate = data_means["vdt"][-1]
    data_means = pd.DataFrame(
        dict(
            stimulus_marker=[int(x) for x in data_means["marker"][:-1]],
            number_bouts=[int(x) for x in data_means["bouts"][:-1]],
            lfp_trace=data_means["lfpNorm"][:-1],
            mean_lfp_trace=data_means["lfpMean"][:-1],
            base_lfp_trace=data_means["b1lfpNorm"][:-1],
            base_mean_lfp_trace=data_means["b1lfpMean"][:-1],
            stimulus_amplitude_modulation=data_means["ampmod"][:-1],
            stimulus_waveform_modulation=data_means["wavmod"][:-1],
            lfp_sampling_rate=[lfp_sampling_rate] * len(data_means["marker"][:-1]),
            lfp_times = [lfp_means_time] * len(data_means["marker"][:-1]),
        )
    )
    data_stats = pd.DataFrame(
        dict(
            stimulus_marker=[int(x) for x in data_stats["marker"][:-1]],
            number_bouts=[int(x) for x in data_stats["bouts"][:-1]],
            lfp_response=data_stats["n1"][:-1],
            base_lfp_response=data_stats["b1n1"][:-1],
            mean_response_modulation=data_stats["aveNn1m"][:-1],
        )
    )
    return pd.merge(data_means, data_stats, on=["stimulus_marker", "number_bouts"])

In [ ]:
for folder in glob('raw/*', recursive=True):
    if os.path.isdir(folder):
        print(folder)
        fnames = glob(f"{folder}/*lfp_means.mat")
        waveforms_fname = glob(f"{folder}/waveform*.mat")
        if len(waveforms_fname) == 1:
            waveforms_fname = waveforms_fname[0]
            waveforms = load_waveform(waveforms_fname)
            fnames.remove(waveforms_fname)
            for fname in fnames:
                print(fname)
                df = load_lfp_data(fname)
                # df = pd.read_csv(fname)
                # df = df.dropna()
                # df = df.drop_duplicates()
                # df.to_csv(fname, index=False)
            print()
        else:
            raise ValueError(f"More than one waveform file in {folder}")